In [8]:
import pennylane as qml
import pennylane.numpy as np

# PyTorch TensorBoard support
from torch.utils.tensorboard import SummaryWriter
from datetime import datetime

In [9]:
EPOCHS = 100
SEED = 16
P = 0.25

In [10]:
def PoolingCirc(weights, wires):
    qml.CRY(weights[0], wires = wires)
    qml.PauliX(wires[0])
    qml.CRY(weights[1], wires = wires)
    qml.PauliX(wires[0])

def ConvCirc1(weights, wires):
    qml.RY(weights[0], wires = wires[0])
    qml.RY(weights[1], wires = wires[1])
    qml.CNOT(wires = wires)

In [11]:
# from ansatz import ConvCirc1, PoolingCirc

inputs = ['+X','-X','+Y','-Y','+Z','-Z']

dev = qml.device("default.mixed", wires = 4)

@qml.qnode(dev, interface='autograd')
def circuit(conv_ansatz, pooling_ansatz, params, data, p):

    if data not in inputs:
        raise ValueError('Invalid Input')

    # Data encoding
    if data == '+X':
        qml.Hadamard(wires=0)
    elif data == '-X':
        qml.PauliX(wires=0)
        qml.Hadamard(wires=0)
    elif data == '+Y':
        qml.RX(-np.pi/2, wires = 0)
    elif data == '-Y':
        qml.RX(np.pi/2, wires=0)
    elif data == '-Z':
        qml.PauliX(wires = 0)

    # Shor Encoding
    qml.CNOT(wires = [0,1])
    qml.CNOT(wires = [0,2])

    #Error Generating
    for qidx in range(3):
        qml.DepolarizingChannel(p, wires=qidx, do_queue=True, id=None)

    #QCNN
    ConvCirc1(params[0:2], wires = [0, 1])  
    ConvCirc1(params[2:4], wires = [2, 3])    
    ConvCirc1(params[4:6], wires = [1, 2])
    ConvCirc1(params[6:8], wires = [3, 0])

    PoolingCirc(params[8:10], wires = [0, 1])
    PoolingCirc(params[10:12], wires = [2, 3])

    ConvCirc1(params[12:14], wires = [1, 3])
    PoolingCirc(params[14:16], wires = [1, 3])

    #adjoint
    if data == '+X':
        qml.adjoint(qml.Hadamard)(wires=0)
    elif data == '-X':
        qml.adjoint(qml.Hadamard)(wires=0)
        qml.adjoint(qml.PauliX)(wires=0)
    elif data == '+Y':
        qml.adjoint(qml.RX)(-np.pi/2, wires = 0)
    elif data == '-Y':
        qml.adjoint(qml.RX)(np.pi/2, wires=0)
    elif data == '-Z':
        qml.adjoint(qml.PauliX)(wires = 0)

    return qml.expval(qml.PauliZ(3))

In [12]:
def cost(params):
    batch = inputs
    conv_ansatz = ConvCirc1
    pooling_ansatz = PoolingCirc
    p = P
    lossvalue = 0
    for data in batch:
        lossvalue += circuit(conv_ansatz, pooling_ansatz, params, data, p)
    return -1 * lossvalue / len(batch)

In [13]:
params = np.random.rand(16, requires_grad=True) # TODO: choose params length
optimizer = qml.AdamOptimizer(stepsize = 0.01)

In [14]:
import logging
import datetime
import os

filepath = './experiment_results'
if os.path.exists(filepath)==False:
    os.makedirs(filepath)

writer = SummaryWriter(log_dir='./QEC_Sim')

for epoch in range(EPOCHS):
    params, loss = optimizer.step_and_cost(cost, params)
    writer.add_scalar('training loss', loss.item(), epoch)
    np.save(filepath+'/'+f'epoch{epoch}', params)
    
    writer.flush()


0
1
2
3
